In [23]:
from napkin import response, request
from pymongo import MongoClient, UpdateOne
import json
import os

CLIENT_NAME = request.path_params.get("CLIENT_NAME")
MONGO_AUTH = os.environ.get("MONGO_AUTH")
body = json.loads(request.data)

In [24]:
from passlib.context import CryptContext

pwd_context = CryptContext(
  schemes=["pbkdf2_sha256"],
  default="pbkdf2_sha256",
  pbkdf2_sha256__default_rounds=30000
)

In [25]:
client = MongoClient("mongodb+srv://{}@meusite.hcxhp.mongodb.net/Site?retryWrites=true&w=majority".format(MONGO_AUTH))
db = client.Site

Coll = db[CLIENT_NAME]
Auth = Coll.find_one({ "_id": "Auth", "users.email": body["email"] })

In [26]:
if Auth is None:
  response.status_code = 401

users = Auth.get("users")
user = [usr for usr in users if usr.get("email") == body["email"]][0]
user

{'email': 'leonardogviva@gmail.com',
 'pass': '$pbkdf2-sha256$30000$RIiRkjJG6J1zTilljFFKyQ$gIdJjCHSDJMg63u2bidgsaZoec34G0SzQJBZ4yz7bX8'}

In [29]:
password_match = pwd_context.verify(body["pass"], user.get("pass"))

response.status_code = 200 if password_match else 501

True